In [1]:
from load_utils import prepare_data
from seq2seq_model import *
import numpy as np
import torch
from tqdm.notebook import tnrange, tqdm_notebook

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
q, a, pairs, vector = prepare_data('test', 'glove.42B.300d/glove.42B.300d.txt', small=True)

Reading test -------
Read 4041 sentence pairs
Counting words
Counted words:
In questions: 6723 words
In answers: 4670 words


In [4]:
matrix_len = q.n_words
weights_matrix = np.zeros((matrix_len, 300))
word_found = 0
for i, word in enumerate(q.word2index):
    try:
        weights_matrix[i] = vector[word]
    except:
        continue

In [5]:
q.word2index

{',': 0,
 'the': 1,
 '.': 2,
 'and': 3,
 'to': 4,
 'of': 5,
 'a': 6,
 'in': 7,
 '"': 8,
 'is': 9,
 'for': 10,
 ':': 11,
 'i': 12,
 ')': 13,
 'that': 14,
 '(': 15,
 'you': 16,
 'it': 17,
 'on': 18,
 '-': 19,
 'with': 20,
 "'s": 21,
 'this': 22,
 'by': 23,
 'are': 24,
 'at': 25,
 'as': 26,
 'be': 27,
 'from': 28,
 'have': 29,
 'was': 30,
 'or': 31,
 'your': 32,
 'not': 33,
 '...': 34,
 'we': 35,
 '!': 36,
 'but': 37,
 '?': 38,
 'all': 39,
 'will': 40,
 'an': 41,
 'my': 42,
 'can': 43,
 'they': 44,
 "n't": 45,
 'do': 46,
 'he': 47,
 'more': 48,
 'if': 49,
 'one': 50,
 'has': 51,
 '|': 52,
 'so': 53,
 'about': 54,
 'new': 55,
 'what': 56,
 'his': 57,
 'there': 58,
 'up': 59,
 'out': 60,
 ';': 61,
 'their': 62,
 'our': 63,
 "'": 64,
 'like': 65,
 'when': 66,
 '$': 67,
 'just': 68,
 'time': 69,
 '&': 70,
 'me': 71,
 'which': 72,
 'who': 73,
 'no': 74,
 'would': 75,
 '/': 76,
 '1': 77,
 'some': 78,
 'get': 79,
 '[': 80,
 ']': 81,
 'also': 82,
 'other': 83,
 'how': 84,
 'may': 85,
 'had': 86,


In [6]:
weights_matrix.shape

(6723, 300)

In [7]:
input = torch.randn(3, 5, requires_grad=True)
target = torch.randn(3, 5)

mae_loss = nn.L1Loss()
loss = mae_loss(input, target)

In [8]:
loss.data.cpu()

tensor(0.6656)

In [9]:
loss.detach().numpy()*32

21.297649383544922

In [10]:
vector = torch.rand(32, 200, dtype=torch.float)
seq_length = np.random.randint(11, size=32)
dec_len = 10

In [11]:
def assign_EOS(vector, batch_size, seq_length, current_step):

    for idx, num in enumerate(seq_length):
        if num < current_step:
            vector[idx, 1] = 100
    return vector


In [12]:
seq_length

array([ 1, 10,  6,  1,  8,  3,  0,  3,  6,  8,  4,  4,  4,  5,  9,  4, 10,
        4, 10,  0,  4,  8,  8, 10,  0,  0, 10,  3,  6, 10,  5,  2])

In [13]:
assign_EOS(vector, 32, seq_length, 2)

tensor([[4.3439e-01, 1.0000e+02, 2.9915e-01,  ..., 5.9393e-01, 7.1220e-01,
         5.0490e-01],
        [1.6428e-01, 2.1585e-01, 3.8403e-01,  ..., 9.4325e-01, 1.1261e-01,
         6.7254e-01],
        [7.2774e-01, 7.6560e-01, 3.2057e-01,  ..., 9.2370e-01, 3.0549e-01,
         2.2987e-01],
        ...,
        [3.8219e-01, 1.8700e-01, 1.2595e-01,  ..., 5.0684e-01, 3.7640e-01,
         5.8411e-01],
        [3.9999e-01, 8.7773e-01, 6.6336e-02,  ..., 1.0167e-01, 9.1080e-01,
         4.0326e-01],
        [7.1180e-01, 1.6704e-01, 4.6169e-01,  ..., 8.5693e-01, 3.6599e-01,
         7.3163e-01]])

In [14]:
true_batch = []
for i in range(0, 32):
    row_list = []
    for word in pairs[i][1].split():
        row_list.append(a.word2index[word])
    true_batch.append(row_list)

In [15]:
true_batch

[[2, 3, 4],
 [5, 6, 7, 8, 9],
 [7, 10, 11, 12, 13, 9],
 [14, 15, 13, 16, 17, 18, 19, 20, 4, 21, 14, 15, 22, 23, 24, 4],
 [25, 26, 27, 28, 29, 30, 10, 31, 9],
 [10, 31, 15, 32, 33, 34, 35, 36, 34, 37, 27, 15, 38, 39, 40, 27],
 [3, 41, 4],
 [42, 43, 13, 44, 45, 46, 9],
 [47, 29, 48, 9, 49, 16, 15, 50, 47, 4],
 [3, 51, 47, 52, 53, 4],
 [3, 54, 55, 18, 56, 14, 15, 57, 4],
 [],
 [58, 59, 24, 60, 32, 61, 17, 62, 63, 9],
 [64, 65, 59, 66, 67, 63, 68, 69, 9, 2, 70, 71, 72, 73, 74, 75, 76, 63, 9],
 [64, 10, 77, 78, 38, 79, 80, 81, 80, 82, 83, 80, 9, 84, 85, 4],
 [5, 86, 9, 87, 63, 43, 26, 41, 88, 29, 89, 9, 90, 74, 42, 4],
 [91, 10, 92, 67, 63, 93, 94, 9],
 [95, 96, 74, 97, 98, 41, 88, 99, 4],
 [28, 29, 98, 27],
 [10,
  100,
  101,
  10,
  100,
  44,
  102,
  103,
  9,
  10,
  100,
  104,
  105,
  106,
  107,
  108,
  59,
  109,
  10,
  16,
  110,
  111,
  41,
  108,
  112,
  113,
  9,
  47,
  43,
  104,
  114,
  26,
  41,
  115,
  68,
  10,
  16,
  116,
  117,
  118,
  119,
  9],
 [10,
  96,
 

In [29]:
true_batch2 = []
for i in range(32, 64):
    row_list = []
    for word in pairs[i][1].split():
        row_list.append(a.word2index[word])
    true_batch2.append(row_list)

In [30]:
true_batch2

[[195, 15, 110, 73, 74, 150, 9],
 [15, 73, 196, 42, 131, 44, 197, 198, 9],
 [26, 199, 200, 9, 10, 16, 44, 201, 202, 9],
 [203,
  10,
  204,
  44,
  205,
  206,
  9,
  99,
  168,
  206,
  43,
  207,
  164,
  9,
  10,
  16,
  44,
  208,
  209,
  9,
  82,
  10,
  100,
  101,
  10,
  210,
  211,
  44,
  212,
  9,
  10,
  213,
  201,
  9],
 [214, 9],
 [3, 11, 10, 215, 32, 14, 216, 4],
 [217, 15, 114, 69, 9],
 [218, 219, 220, 82, 221, 107, 222, 223, 9],
 [224, 225, 226, 9],
 [10, 227, 57, 228, 229, 230, 45, 231, 82, 232, 9],
 [7, 96, 74, 233, 10, 73, 14, 234, 9],
 [235, 15, 236, 237, 47, 238, 239, 113, 9],
 [],
 [240, 27, 26, 32, 241, 24, 15, 9, 49, 38, 15, 4],
 [242, 38, 243, 58, 107, 138, 9],
 [240,
  10,
  96,
  74,
  141,
  47,
  42,
  29,
  44,
  26,
  244,
  32,
  98,
  245,
  246,
  9,
  10,
  13,
  233,
  32,
  247,
  151,
  248,
  249,
  82,
  250,
  32,
  251,
  252,
  253,
  9],
 [10,
  73,
  74,
  13,
  254,
  107,
  84,
  255,
  154,
  99,
  235,
  10,
  256,
  108,
  257,
  258

In [16]:
error = 0
for b, (pred_seq, gt_seq) in enumerate(zip(true_batch, true_batch)):
    if pred_seq != gt_seq:
        error += 1

In [17]:
error

0

    MAKE SURE THAT PREDICT METHOD RETURNS A LIST OF LITS OF INDEXES LIKE THE ABOVE!!!!!!!!!!!!

In [18]:
# Declaire optimizer, batch, etc...

# Initiate Model


In [31]:
true_batch.extend(true_batch2)

In [33]:
len(true_batch)

64

In [47]:
references = [[[a.index2word[idx] for idx in line]] for line in true_batch]

In [48]:
hypotheses = [[a.index2word[idx] for idx in line] for line in true_batch]

In [49]:
from nltk.translate import bleu_score

In [50]:
 bleu = bleu_score.corpus_bleu(
            references, hypotheses,
            smoothing_function=bleu_score.SmoothingFunction().method1)

In [51]:
bleu

0.9903514614508018

In [ ]:
# Now run for 100 epochs
for epoch in tnrange(100, desc="Total epochs: "):
  
    # Calculte loss
    loss = epoch_train(model, optimizer, batch_size, pairs, device)

    # Calculate accuracy
    

    # Try to do interactive plot
    

    

print(f"Optimization ended successfully")